# 4.2 Final Models: Cross-Browser Evaluation


In this notebook we cross-browser evaluate the finalized GCN and GraphSAGE models:

- GCN/GraphSAGE Model
    - Cross Browser Validation

### Dataset Configuration

In [1]:
data_name = 'chrome-run-01'
with_centrality_metrics = False

### Set Up

In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import torch
import os

In [4]:
import sys
sys.path.append(os.path.join('..', 'models', 'definitions'))
sys.path.append(os.path.join('..', 'src', 'models'))
import evaluate_model

For the evaluation, we load our ```trained``` GNN models and the ```processed``` dataset.

In [5]:
# Load the model and data
org_data_name = data_name
if with_centrality_metrics:
    data_name = f"{data_name}-with-centrality-metrics"
else:
    data_name = f"{data_name}-without-centrality-metrics"

data_folder = os.path.join('..', 'data', 'processed')
data_path = os.path.join(data_folder, f"{data_name}.pt")
gcn_path = os.path.join('..', 'models', 'trained', 'gcn.pt')
graphsage_path = os.path.join('..', 'models', 'trained', 'graphsage.pt')

data = torch.load(data_path)
gcn = torch.load(gcn_path)
graphsage = torch.load(graphsage_path)

We load one dataset of each different browser from the ```processed``` file:

In [6]:
# Define configuration to load other datasets
data_parts = data_name.split("-")
run = data_parts[2]
current_browser = data_parts[0]
file_ending = '-with-centrality-metrics.pt' if with_centrality_metrics else '-without-centrality-metrics.pt'
files = files = os.listdir(data_folder)

# Construct the datasets list
browsers = []

# Helper function to match the desired pattern
def is_other_browser(file, browser, file_ending):
    return not file.startswith(browser)  and file.endswith(file_ending)

# Filter files based on the user's input and the specified ending
for file in files:
    if is_other_browser(file, current_browser, file_ending):
        # Extract the name part before the ending
        name = file.split("-with", 1)[0]
        # Load the data using torch.load
        data_path = os.path.join(data_folder, file)
        data = torch.load(data_path)
        # Append the dictionary to the datasets list
        browsers.append({'name': name, 'data': data})

# Print the resulting datasets
print(f"Other Browsers:\n{browsers}")

Other Browsers:
[{'name': 'brave-run-01', 'data': Data(x=[15022, 33], edge_index=[2, 32012], num_nodes=15022, y=[15022], train_mask=[15022], val_mask=[15022], test_mask=[15022])}, {'name': 'firefox-run-01', 'data': Data(x=[25138, 33], edge_index=[2, 124924], num_nodes=25138, y=[25138], train_mask=[25138], val_mask=[25138], test_mask=[25138])}]


### GCN Model

#### Cross Browser Evaluation

In [7]:
cross_browser_gcn = os.path.join('..', 'results', 'gcn', 'cross_browser_evaluation.csv')

evaluate_model.datasets_evaluate(browsers, gcn, cross_browser_gcn)


Evaluating GCN_2_Layers on different datasets...
Dataset: brave-run-01, Accuracy: 0.8666, Precision: 0.0893, Recall: 0.5916, F1 Score: 0.1551
Dataset: firefox-run-01, Accuracy: 0.8077, Precision: 0.6584, Recall: 0.8172, F1 Score: 0.7293
Successfully evaluated GCN_2_Layers on different datasets


### GraphSAGE Model

#### Cross Browser Evaluation

In [8]:
cross_browser_graphsage = os.path.join('..', 'results', 'variant', 'cross_browser_evaluation.csv')

evaluate_model.datasets_evaluate(browsers, graphsage, cross_browser_graphsage)


Evaluating GraphSAGE on different datasets...
Dataset: brave-run-01, Accuracy: 0.8657, Precision: 0.1083, Recall: 0.7588, F1 Score: 0.1896
Dataset: firefox-run-01, Accuracy: 0.8286, Precision: 0.6906, Recall: 0.8319, F1 Score: 0.7547
Successfully evaluated GraphSAGE on different datasets
